# Project: Monitoring NDVI Changes and Segmenting Lost Vegetation

## 1. Introduction
**Objective**: Monitor the changes in NDVI over the years and segment areas of lost vegetation to classify what they have been converted to using the EuroSAT dataset.

**Tools**: Python, Satellite Imagery, NDVI Calculation, EuroSAT Dataset, Machine Learning Models

## 2. Data Collection and Preparation
### Step 1: Collect Satellite Images
- **Sources**: Landsat, Sentinel-2
- **Period**: Define the years for monitoring (e.g., 2000-2020)
- **Regions**: Define the geographic regions of interest

### Step 2: Preprocess Satellite Images
- **Cloud Masking**: Remove clouds and shadows
- **Radiometric Calibration**: Correct sensor noise and inconsistencies

### Step 3: Calculate NDVI
- **Formula**: NDVI = (NIR - RED) / (NIR + RED)
- **Implementation**: Use libraries like `rasterio` and `numpy`

### Step 4: Load EuroSAT Dataset
- **Download**: Obtain the EuroSAT dataset
- **Categories**: Urban, Agriculture, Forest, etc.

## 3. NDVI Analysis
### Step 1: Compute Yearly NDVI Maps
- **Mean NDVI**: Calculate average NDVI for each year
- **Seasonal NDVI**: If needed, calculate for different seasons

### Step 2: Identify Changes in NDVI
- **Trend Analysis**: Use statistical methods to identify significant changes over the years
- **Change Detection**: Highlight areas with significant NDVI reduction

## 4. Segmentation of Lost Vegetation
### Step 1: Identify Lost Vegetation Tiles
- **Thresholding**: Define NDVI thresholds to identify loss
- **Mask Creation**: Create masks for areas of NDVI reduction

### Step 2: Extract Lost Vegetation Tiles
- **Coordinate Extraction**: Get coordinates of identified tiles
- **Tile Extraction**: Extract these regions from satellite images

## 5. Classification Using EuroSAT
### Step 1: Data Preparation
- **Tile Labelling**: Label extracted tiles according to EuroSAT categories
- **Data Augmentation**: If necessary, augment data to improve model performance

### Step 2: Model Selection and Training
- **Model**: Choose a suitable classification model (e.g., CNN)
- **Training**: Train the model using EuroSAT dataset
- **Validation**: Validate model performance on a separate validation set

### Step 3: Classification of Lost Vegetation Tiles
- **Prediction**: Use the trained model to classify lost vegetation tiles
- **Post-processing**: Aggregate and interpret classification results

## 6. Results and Analysis
### Step 1: Visualization
- **NDVI Changes**: Visualize NDVI changes over the years using plots or maps
- **Classified Tiles**: Map out classified lost vegetation tiles

### Step 2: Interpretation
- **Land Use Changes**: Interpret what the lost vegetation has been converted to
- **Environmental Impact**: Discuss potential environmental impacts

## 7. Conclusion
- **Summary**: Summarize key findings
- **Future Work**: Suggest possible extensions or improvements

## 8. References and Documentation
- **References**: List of scholarly articles, datasets, and tools used
- **Documentation**: Detailed code comments and project documentation


In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import IPython.display as disp

%matplotlib inline

# Dataset

In [2]:
import ee

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize()
